# Tokenization compression ratios

In [1]:
import requests
from urllib.parse import urlparse

import string
import tiktoken

In [2]:
# GPT-4's Tokenizer
tokenizer = tiktoken.get_encoding('cl100k_base')

# get the books

In [3]:
# all books have the same url format;
# they are unique by numerical code
baseurl = 'https://www.gutenberg.org/cache/epub/'

bookurls = [
    # code       title
    ['84',    'Frankenstein'    ],
    ['64317', 'GreatGatsby'     ],
    ['11',    'AliceWonderland' ],
    ['1513',  'RomeoJuliet'     ],
    ['76',    'HuckFinn'        ],
    ['219',   'HeartDarkness'   ],
    ['2591',  'GrimmsTales'     ],
    ['2148',  'EdgarAllenPoe'   ],
    ['36',    'WarOfTheWorlds'  ],
    ['829',   'GulliversTravels']
]

In [4]:
print('  Book title     |  Chars  |  Tokens | Compression')
print('-'*50)

for code, title in bookurls:
    # get the text
    fullurl = baseurl + code + '/pg' + code + '.txt'
    text = requests.get(fullurl).text
    num_chars = len(text)

    # tokenize
    tokens = tokenizer.encode(text)
    num_tokens = len(tokens)

    # compression ratio
    compress = (num_tokens / num_chars) * 100

    print(f"{title:16} | {num_chars:>7,d} | {num_tokens:>7,d} | {compress:>3.2f}%")


  Book title     |  Chars  |  Tokens | Compression
--------------------------------------------------
Frankenstein     | 446,544 | 102,419 | 22.94%
GreatGatsby      | 296,858 |  70,343 | 23.70%
AliceWonderland  | 167,674 |  41,457 | 24.72%
RomeoJuliet      | 167,429 |  43,761 | 26.14%
HuckFinn         | 602,714 | 159,125 | 26.40%
HeartDarkness    | 232,885 |  56,483 | 24.25%
GrimmsTales      | 549,736 | 137,252 | 24.97%
EdgarAllenPoe    | 632,131 | 144,315 | 22.83%
WarOfTheWorlds   | 363,420 |  84,580 | 23.27%
GulliversTravels | 611,742 | 143,560 | 23.47%


# let's see some websites

In [5]:
weburls = [
    'http://python.org/',
    'https://pytorch.org/',
    'https://en.wikipedia.org/wiki/List_of_English_words_containing_Q_not_followed_by_U',
    'https://sudoku.com/',
    'https://reddit.com/',
    'https://visiteurope.com/en/',
    'https://sincxpress.com/',
    'https://openai.com/',
    'https://theuselessweb.com/',
    'https://maps.google.com/',
    'https://pigeonsarentreal.co.uk/',
]

In [6]:
print('    Website        |  Chars  |  Tokens | Compression')
print('-'*53)

for url in weburls:

    # get the text
    text = requests.get(url).text
    num_chars = len(text)

    # tokenize
    tokens = tokenizer.encode(text)
    num_tokens = len(tokens)

    # compression ratio
    compress = 100*num_tokens/num_chars

    print(f'{urlparse(url).hostname[:-4]:18} | {num_chars:>7,d} | {num_tokens:>7,d} |  {compress:>3.2f}%')

    Website        |  Chars  |  Tokens | Compression
-----------------------------------------------------
python             |  50,332 |  12,791 |  25.41%
pytorch            | 388,061 | 111,398 |  28.71%
en.wikipedia       |      92 |      26 |  28.26%
sudoku             | 139,673 |  51,094 |  36.58%
reddit             | 479,437 | 148,207 |  30.91%
visiteurope        | 116,010 |  31,668 |  27.30%
sincxpress         |  25,580 |   6,843 |  26.75%
openai             |  11,617 |   6,461 |  55.62%
theuselessweb      |   4,756 |   1,329 |  27.94%
maps.google        | 212,313 | 107,456 |  50.61%
pigeonsarentreal.c | 243,854 |  71,232 |  29.21%


# using the 'string' library

In [8]:
print('  Attribute     |  Chars  |  Tokens | Compression')
print('-'*50)

for k,v in string.__dict__.items():
    if isinstance(v,str) and (len(v)>0):

        # get the text
        num_chars = len(v)

        # tokenize
        tokens = tokenizer.encode(v)
        num_tokens = len(tokens)

        # compression ratio
        compress = 100*num_tokens/num_chars

        print(f'{k:15} | {num_chars:>7,d} | {num_tokens:>7,d} |  {compress:>5.2f}%')

  Attribute     |  Chars  |  Tokens | Compression
--------------------------------------------------
__name__        |       6 |       1 |  16.67%
__doc__         |     622 |     109 |  17.52%
__package__     |       6 |       1 |  16.67%
__file__        |      84 |      22 |  26.19%
__cached__      |     109 |      31 |  28.44%
whitespace      |       6 |       4 |  66.67%
ascii_lowercase |      26 |       1 |   3.85%
ascii_uppercase |      26 |       1 |   3.85%
ascii_letters   |      52 |       2 |   3.85%
digits          |      10 |       4 |  40.00%
hexdigits       |      22 |       7 |  31.82%
octdigits       |       8 |       3 |  37.50%
punctuation     |      32 |      21 |  65.62%
printable       |     100 |      31 |  31.00%


In [9]:
# e.g.,
string.__dict__['__doc__']
# string.ascii_lowercase

'A collection of string constants.\n\nPublic module variables:\n\nwhitespace -- a string containing all ASCII whitespace\nascii_lowercase -- a string containing all ASCII lowercase letters\nascii_uppercase -- a string containing all ASCII uppercase letters\nascii_letters -- a string containing all ASCII letters\ndigits -- a string containing all ASCII decimal digits\nhexdigits -- a string containing all ASCII hexadecimal digits\noctdigits -- a string containing all ASCII octal digits\npunctuation -- a string containing all ASCII punctuation characters\nprintable -- a string containing all ASCII characters considered printable\n\n'

# Tokenization in different languages

Tokenization ≠ compression (but it often is).

Languages that have more complex written forms (e.g., morphemes in Chinese, richer morphology in Tamil) may require more tokens that characters.

Tokenization is less effective in languages they have less training data on.

In [10]:
from transformers import BertTokenizer
tokenizerB = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
languages = ['English','Hindi','Spanish','Arabic','Persian','Lithuanian','Chinese','Tamil','Esperanto']

sentences = [ 'Blue towels are great because they remind you of the sea, although the sea is wet and towels work better when they are dry.',
              'नीले तौलिए बहुत अच्छे होते हैं क्योंकि वे आपको समुद्र की याद दिलाते हैं, हालांकि समुद्र गीला होता है और तौलिए सूखे होने पर बेहतर काम करते हैं।',
              'Las toallas azules son geniales porque recuerdan al mar, aunque el mar está mojado y las toallas funcionan mejor cuando están secas.',
              'تعتبر المناشف الزرقاء رائعة لأنها تذكرك بالبحر، على الرغم من أن البحر مبلل والمناشف تعمل بشكل أفضل عندما تكون جافة.',
              'حوله‌های آبی عالی هستند زیرا شما را به یاد دریا می‌اندازند، اگرچه دریا مرطوب است و حوله‌ها وقتی خشک باشند بهتر عمل می‌کنند.',
              'Mėlyni rankšluosčiai puikūs, nes primena jūrą, nors jūra yra šlapia, o rankšluosčiai geriau tinka, kai yra sausi.',
              '蓝色毛巾很棒，因为它们会让您想起大海，尽管海水是湿的，而毛巾在干燥时效果更好。',
              'நீல நிற துண்டுகள் சிறந்தவை, ஏனென்றால் அவை கடலை நினைவூட்டுகின்றன, இருப்பினும் கடல் ஈரமாக இருக்கும், துண்டுகள் உலர்ந்திருக்கும் போது சிறப்பாக வேலை செய்யும்.',
              'Bluaj mantukoj estas bonegaj ĉar ili memorigas vin pri la maro, kvankam la maro estas malseka kaj mantukoj funkcias pli bone kiam ili estas sekaj.',
]

In [14]:
# table header
print(' Language  |  Chars  |  BERT  |  GPT ')
print('-'*37)

for lang, text in zip(languages, sentences):
    # tokenize the text
    tokens_g = tokenizer.encode(text)
    tokens_b = tokenizerB.encode(text)[1:-1]

    print(f"{lang:>10} |   {len(text):3}   |  {len(tokens_b):3}   |  {len(tokens_g):3}")

 Language  |  Chars  |  BERT  |  GPT 
-------------------------------------
   English |   123   |   26   |   26
     Hindi |   142   |   78   |  136
   Spanish |   132   |   46   |   34
    Arabic |   115   |   95   |   84
   Persian |   123   |   96   |   90
Lithuanian |   113   |   46   |   57
   Chinese |    39   |   39   |   55
     Tamil |   154   |   35   |  209
 Esperanto |   146   |   56   |   50
